# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import scipy.stats
import math

In [16]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [18]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [19]:
print(data.columns)

Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')


In [20]:
print(data.race.size)

4870


<b>Question 1: What test is appropriate for this problem? Does CLT apply?<p></b>
Two-sample t-test is appropiate because we are trying to determine if two population means are equal.</p><p>
CLT applies because our sample size is larger than 30, and observations can be considered independent.</p><b><p>
Question 2: What are the null and alternate hypotheses?</b></p><p>
    Null Hypothesis: There is no difference between call back rates for black and white resumes. </p><p>
    Alternative Hypothesis: There is a difference between call back rates for black and white resumes. 

<b>Question 3: Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.</b>

In [34]:
data = pd.concat([data.race,data.call], axis=1)
print(data.head())

  race  call
0    w   0.0
1    w   0.0
2    b   0.0
3    b   0.0
4    w   0.0


In [63]:
b = len(data[data['race'] == 'b']) # black sample size
w = len(data[data['race'] == 'w']) # white sample size
b, w

(2435, 2435)

In [69]:
# number of callbacks for black and white-sounding names
b_call = sum(data[data.race=='b'].call)
w_call = sum(data[data.race=='w'].call)

In [73]:
#callback rates for black and white-sounding names
b_rate = (b_call/b)
w_rate = (w_call/w)

In [72]:
# conduct two-sample t-test

p = (b*b_rate+w*w_rate)/(b+w) # joint callback
std= math.sqrt(p*(1-p)*((1/b)+(1/w))) # joint standard deviation

test_stat = (b_rate-w_rate)/std
p_val = stats.t.sf(abs(test_stat), df=len(data)-1)*2

print('test statistic:',test_stat)
print('p value:',p_val)

test statistic: -4.108412152434346
p value: 4.0493178875903686e-05


In [66]:
#variance
var_b = (b_rate*(1-b_rate)/b)
var_b

2.4771737856498466e-05

In [67]:
var_w = w_rate*(1-w_rate)/w
var_w

3.580911983304638e-05

In [85]:
# calculate margins of error for the two distributions
b_std_err = math.sqrt(var_b) # black standard error
w_std_err = math.sqrt(var_w) # white standard error
stderr_diff = math.sqrt((b_std_err**2 + w_std_err**2)) # standard error for two samples
print(stderr_diff)

0.0077833705866767544


In [86]:
sample_diff = abs(b_rate-w_rate)

In [88]:
#margin of error
moe = 1.96*stderr_diff
moe

0.015255406349886438

In [89]:
#confidence interval
conf_int_min = sample_diff - moe
conf_int_max = sample_diff + moe
print(conf_int_min, conf_int_max)

0.016777447859559147 0.047288260559332024


The margin of error is 0.015255406349886438. The 95% confidence interval is: [0.016777447859559147, 0.047288260559332024]. And the p-value is 4.0493178875903686e-05

<b>Question 4: Write a story describing the statistical significance in the context or the original problem.</b>

The obtained p-value is extremely lower than 0.05, which means we can reject the null hypothesis, and accept the alternative hypothesis that states there is a difference between call backs for black and white resumes. 

<b>Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</b>

No, my analysis shows that race is a factor in callback success, not necessarily the most important. In order to amend my analysis, I would need to compare every column (listed above) with callback rate success in order to rule out/in other factors that may impact overall callback success. 